In [ ]:
# Impression 클릭한 것 -1인애들만 남기기기

import pandas as pd

# 1. Load train + dev behaviors
train_behaviors = pd.read_csv(
    'download/MINDsmall_train/behaviors.tsv',
    sep='\t',
    names=['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']
)
dev_behaviors = pd.read_csv(
    'download/MINDsmall_dev/behaviors.tsv',
    sep='\t',
    names=['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']
)

# 2. Combine train and dev
combined_behaviors = pd.concat([train_behaviors, dev_behaviors], ignore_index=True)

# 3. Remove impressions ending with -0
def remove_zero_impressions(impressions):
    items = str(impressions).split()
    return " ".join(item for item in items if not item.endswith("-0"))

combined_behaviors['Impressions'] = combined_behaviors['Impressions'].apply(remove_zero_impressions)

# (Optional) Save the cleaned result
combined_behaviors.to_csv('behaviors_filtered.csv', sep='\t', index=False)


In [ ]:
# 클릭한 애들이 여러개 인경우 행 분리하기
import pandas as pd

# 1. Load train + dev behaviors
train_behaviors = pd.read_csv(
    'download/MINDsmall_train/behaviors.tsv',
    sep='\t',
    names=['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']
)
dev_behaviors = pd.read_csv(
    'download/MINDsmall_dev/behaviors.tsv',
    sep='\t',
    names=['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']
)

# 2. Combine train and dev
combined = pd.concat([train_behaviors, dev_behaviors], ignore_index=True)

# 3. Remove impressions ending with -0
def remove_zero(imps):
    return " ".join(i for i in str(imps).split() if not i.endswith("-0"))

combined['Impressions'] = combined['Impressions'].apply(remove_zero)

# 4. Split rows where Impressions > 1
expanded_rows = []

for _, row in combined.iterrows():
    impressions = row['Impressions'].split()
    if len(impressions) <= 1:
        expanded_rows.append(row)
    else:
        for imp in impressions:
            new_row = row.copy()
            new_row['Impressions'] = imp
            expanded_rows.append(new_row)

# 5. Final expanded DataFrame
expanded_df = pd.DataFrame(expanded_rows)

# 6. Save if needed
expanded_df.to_csv('expanded_behaviors.csv', sep='\t', index=False)


In [22]:
import pandas as pd

# 1. Load train + dev behaviors
train_behaviors = pd.read_csv(
    'download/MINDsmall_train/behaviors.tsv',
    sep='\t',
    names=['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']
)
dev_behaviors = pd.read_csv(
    'download/MINDsmall_dev/behaviors.tsv',
    sep='\t',
    names=['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']
)

# 2. Combine
combined = pd.concat([train_behaviors, dev_behaviors], ignore_index=True)

# 3. Extract only -0 impressions
zero_rows = []

for _, row in combined.iterrows():
    impressions = str(row['Impressions']).split()
    for imp in impressions:
        if imp.endswith("-0"):
            new_row = row.copy()
            new_row['Impressions'] = imp
            zero_rows.append(new_row)

# 4. Create DataFrame
zero_df = pd.DataFrame(zero_rows)



In [ ]:
import pandas as pd
import random

# Load expanded_df (Impressions 1개씩 확장된 것)
expanded_df = pd.read_csv('expanded_behaviors.tsv', sep='\t')

# 1. zero_df에서 20개 무작위 샘플 선택
zero_sample = zero_df.sample(n=20, random_state=42).reset_index(drop=True)

# 2. expanded_df에서 임의의 20개 행 선택
target_indices = random.sample(range(len(expanded_df)), 20)

# 3. 선택된 위치에 zero_sample의 Impressions 값 덮어쓰기
for i, idx in enumerate(target_indices):
    expanded_df.at[idx, 'Impressions'] = zero_sample.at[i, 'Impressions']

# 4. 저장
expanded_df.to_csv('expanded_behaviors22.csv', sep='\t', index=False)


In [ ]:
# 성공 클릭 1인애들 행 분리리
import pandas as pd

# 상수 정의
BEHAVIOR_COLUMNS = ['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']

# 클릭 여부가 1인 인상만 남기고 행을 분리하는 함수
def filter_and_expand_clicks(train_path: str, dev_path: str, output_path: str):
    # 1. 데이터 로드
    train = pd.read_csv(train_path, sep='\t', names=BEHAVIOR_COLUMNS, header=0)
    dev = pd.read_csv(dev_path, sep='\t', names=BEHAVIOR_COLUMNS, header=0)
    df = pd.concat([train, dev], ignore_index=True)

    # 2. 클릭된 인상만 남기기 (클릭값이 -1인 경우)
    df['Impressions'] = df['Impressions'].astype(str).apply(
        lambda imps: " ".join(imp for imp in imps.split() if imp.endswith('-1'))
    )

    # 3. 클릭 인상이 하나라도 있는 행만 유지
    df = df[df['Impressions'].str.strip() != '']

    # 4. 여러 클릭 인상이 있는 경우 행 분리
    expanded_rows = []
    for _, row in df.iterrows():
        impressions = row['Impressions'].split()
        for imp in impressions:
            new_row = row.copy()
            new_row['Impressions'] = imp
            expanded_rows.append(new_row)

    # 5. 최종 데이터프레임 생성 및 저장
    expanded_df = pd.DataFrame(expanded_rows)
    expanded_df.to_csv(output_path, sep='\t', index=False)

if __name__ == '__main__':
    filter_and_expand_clicks(
        'download/MINDsmall_train/behaviors.tsv',
        'download/MINDsmall_dev/behaviors.tsv',
        'expanded_behaviors.tsv'
    )


In [ ]:
# 성공 클릭 0인 애들 20개 랜덤 분배
import pandas as pd
import random

# 상수 정의
BEHAVIOR_COLUMNS = ['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']

if __name__ == '__main__':
    # 1. train/dev behaviors 로드
    train = pd.read_csv('download/MINDsmall_train/behaviors.tsv', sep='\t', names=BEHAVIOR_COLUMNS, header=0)
    dev = pd.read_csv('download/MINDsmall_dev/behaviors.tsv', sep='\t', names=BEHAVIOR_COLUMNS, header=0)
    combined = pd.concat([train, dev], ignore_index=True)

    # 2. 클릭되지 않은 (-0) 인상만 추출
    zero_rows = []
    for _, row in combined.iterrows():
        impressions = str(row['Impressions']).split()
        for imp in impressions:
            if imp.endswith("-0"):
                new_row = row.copy()
                new_row['Impressions'] = imp
                zero_rows.append(new_row)
    zero_df = pd.DataFrame(zero_rows)

    # 중복 제거
    unique_zero_impressions = zero_df['Impressions'].drop_duplicates().tolist()

    if len(unique_zero_impressions) < 20:
        raise ValueError("고유한 -0 인상이 20개 이상 필요합니다.")

    # 3. 확장된 클릭된 인상 불러오기
    expanded_df = pd.read_csv('expanded_behaviors.csv', sep='\t')

    # 4. expanded_df 전체에 대해 각 행마다 고유한 무작위 20개 zero 인상 추가
    for i in range(len(expanded_df)):
        sampled_zeros = random.sample(unique_zero_impressions, 20)
        old_val = expanded_df.at[i, 'Impressions']
        old_count = len(str(old_val).split())
        new_val = f"{old_val} {' '.join(sampled_zeros)}"
        new_count = len(new_val.split())
        expanded_df.at[i, 'Impressions'] = new_val
        if i < 3:
            print(f"[행 {i}] 기존 수: {old_count}, 추가 후 수: {new_count}")
            print(f"추가된 샘플: {' '.join(sampled_zeros)}")
    # 5. 저장
    expanded_df.to_csv('new_behaviors.csv', sep='\t', index=False)


[행 0] 기존 수: 1, 추가 후 수: 21
추가된 샘플: N19915-0 N64083-0 N13288-0 N57679-0 N36683-0 N11470-0 N19431-0 N57095-0 N30528-0 N25822-0 N2669-0 N30890-0 N60834-0 N55633-0 N49892-0 N3469-0 N64797-0 N59895-0 N38388-0 N36975-0
[행 1] 기존 수: 1, 추가 후 수: 21
추가된 샘플: N53114-0 N22111-0 N42486-0 N63369-0 N30803-0 N64699-0 N61838-0 N6749-0 N59821-0 N24813-0 N7806-0 N13408-0 N43090-0 N14744-0 N27168-0 N48350-0 N49308-0 N53808-0 N59211-0 N41972-0
[행 2] 기존 수: 1, 추가 후 수: 21
추가된 샘플: N17087-0 N59857-0 N32890-0 N10785-0 N51585-0 N7811-0 N23758-0 N36448-0 N55604-0 N31986-0 N39466-0 N41304-0 N35625-0 N30367-0 N34048-0 N47454-0 N33134-0 N26452-0 N60364-0 N21646-0


In [48]:
# filter_expand_news.py
import pandas as pd
import random
from datetime import datetime, timedelta

BEHAVIOR_COLUMNS = ['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']
DATE_FORMAT = "%m/%d/%Y %I:%M:%S %p"

if __name__ == '__main__':
    # 1. 뉴스별 publish_time 로드 및 end_time 계산
    publish_df = pd.read_csv('news_publish_times.csv')
    publish_df['publish_time'] = pd.to_datetime(publish_df['publish_time'])
    publish_df['end_time'] = publish_df['publish_time'] + timedelta(hours=24)

    # 2. expanded_behaviors.csv 로드
    expanded_df = pd.read_csv('expanded_behaviors.csv', sep='\t')
    expanded_df['Time'] = pd.to_datetime(expanded_df['Time'], format=DATE_FORMAT, errors='coerce')
    expanded_df.dropna(subset=['Time'], inplace=True)

    # 3. 각 row별로 end_time > Time 인 news_id만 필터링하여 20개 랜덤 샘플링
    for i in range(len(expanded_df)):
        row_time = expanded_df.at[i, 'Time']

        valid_news_df = publish_df[publish_df['end_time'] > row_time]
        valid_news = valid_news_df['news_id'].tolist()

        if len(valid_news) < 20:
            sampled = valid_news
        else:
            sampled = random.sample(valid_news, 20)

        # 기존 Impressions + 샘플 추가
        old_val = str(expanded_df.at[i, 'Impressions'])
        new_val = f"{old_val} {' '.join([f'{nid}-0' for nid in sampled])}"
        expanded_df.at[i, 'Impressions'] = new_val

    # 4. 저장
    expanded_df.to_csv('news_behaviors.csv', sep='\t', index=False)
